# TAIL-RISK FUND: OPTIONS

author: [@sparshsah](https://github.com/sparshsah)

ref: https://github.com/sparshsah/foggy-demo/blob/main/demo/finance/tail-risk-fund.md

In [17]:
# https://github.com/sparshsah/foggy-statslib
import sys; sys.path.append("../../../foggy-statslib/")
import foggy_statslib.core as fsc
import foggy_statslib.fin as fsf

## Setting

You hold 2 shares of S, which is currently trading at 100USD.
(S is SPX, but I don't want the fact that SPX is not tradable and also actually at 3000USD to distract you.)

You believe that it is equally likely to:
* Fall 20pct over the next 6 months, Then stay flat to EOY
* Stay flat over the next 6 months, Then fall 20pct by EOY
* Stay flat through EOY

Prevailing option-implied vols are around 16pct.

In [19]:
s_t = 100
x = 0.20
sigma = 0.16

## Question

Should you
* Purchase four 6-month puts?
* Purchase four 1-year puts?
* Purchase two of each?

In [ ]:
fsf.calc_bsf_option_value(